In [11]:
import pandas as pd
import numpy as np

from Models.grid_search_utils import GridSearch as Gs
from pandas import DataFrame

In [12]:
LGBMRanker = Gs.load_model(name="../Models/BlackBox/lmart/saved_models/LGBMRanker")

In [13]:
FIGSReg = Gs.load_model(name="../Models/GlassBox/figs/saved_models/FIGSReg")

In [14]:
EBMReg = Gs.load_model(name="../Models/GlassBox/ebm/saved_models/EBMReg")

In [15]:
def predict_rank(model, X:DataFrame, top_k:int=15):
    y_pred = model.predict(np.asarray(X.iloc[:,5:].values))
    y_pred = pd.DataFrame(y_pred, index=X.index, columns=["lambdas"])
    dt_final = pd.merge(X, y_pred, left_index=True, right_index=True)
    return dt_final.sort_values("lambdas",ascending=False)[["kId","lambdas","w_score","relevance"]].head(top_k)

In [16]:
test_set = pd.read_csv("../outputs/scores/0_dataset_ts.csv")

In [17]:
test_set[test_set["relevance"]==5]

,qId,kId,score,w_score,relevance,fitness_edu_basic,fitness_edu_bonus,fitness_city,fitness_age,fitness_exp_basic,fitness_exp_bonus,fitness_lang_basic,fitness_lang_bonus,fitness_comp_basic,fitness_comp_bonus,fitness_know_basic,fitness_knowl_bonus
14,117,9401,7.015625,0.717811,5,1.0,0.00,0.515625,1.0,1.0,0.00,1.0,0.0,1.000000,0.500000,1.000000,0.000000
16,88,7157,6.315322,0.711491,5,1.0,0.00,0.248355,1.0,1.0,0.00,1.0,0.0,1.000000,0.017127,1.000000,0.049840
26,21,1758,7.471413,0.726605,5,1.0,0.25,0.984375,1.0,1.0,0.25,1.0,0.0,1.000000,0.077935,0.630204,0.278899
34,95,7689,7.845580,0.759724,5,1.0,0.25,0.829770,1.0,1.0,0.25,1.0,0.0,1.000000,0.015810,1.000000,0.500000
55,27,2227,6.382300,0.770424,5,1.0,0.00,0.847862,1.0,0.0,0.00,1.0,0.0,1.000000,0.034439,1.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,70,5742,7.107969,0.722846,5,1.0,0.00,0.824013,1.0,1.0,0.25,1.0,0.0,1.000000,0.018567,1.000000,0.015388
3149,94,7584,7.731784,0.718890,5,1.0,0.00,0.970395,1.0,1.0,0.25,1.0,0.0,1.000000,0.500000,0.964286,0.047104
3155,24,1983,6.948688,0.710037,5,1.0,0.00,0.693257,1.0,1.0,0.00,1.0,0.0,0.952386,0.034023,1.000000,0.269022
3157,16,1346,6.035314,0.716721,5,1.0,0.00,0.509868,1.0,0.0,0.00,1.0,0.0,1.000000,0.025446,1.000000,0.500000


In [18]:
job = test_set[test_set["qId"]==88]

# Perfect rank

In [19]:
job[["kId","w_score","relevance"]].sort_values("w_score",ascending=False).head(15)

,kId,w_score,relevance
898,7105,0.755986,5
16,7157,0.711491,5
2421,7104,0.701907,4
2607,7161,0.677290,4
229,7113,0.653243,4
2558,7130,0.633010,4
413,7143,0.623950,4
1789,7138,0.614451,4
3090,7142,0.604222,4
372,7110,0.592233,4


# LGBMRanker

In [20]:
predict_rank(LGBMRanker, job)

,kId,lambdas,w_score,relevance
898,7105,9.844129,0.755986,5
2873,7159,9.592338,0.574789,4
3090,7142,9.215515,0.604222,4
16,7157,9.169757,0.711491,5
2607,7161,8.825334,0.677290,4
1789,7138,8.769664,0.614451,4
2431,7150,8.739263,0.569684,4
229,7113,8.686450,0.653243,4
2421,7104,8.663812,0.701907,4
193,7158,5.225820,0.567148,4


# EBM Regressor

In [21]:
predict_rank(EBMReg, job)

,kId,lambdas,w_score,relevance
2607,7161,0.662992,0.677290,4
229,7113,0.660458,0.653243,4
898,7105,0.640526,0.755986,5
2431,7150,0.640319,0.569684,4
2421,7104,0.637372,0.701907,4
2873,7159,0.637308,0.574789,4
3090,7142,0.633874,0.604222,4
16,7157,0.629465,0.711491,5
1964,7147,0.612311,0.580760,4
1789,7138,0.609421,0.614451,4


# FIGSRegressor

In [22]:
predict_rank(FIGSReg, job)

,kId,lambdas,w_score,relevance
16,7157,0.638527,0.711491,5
898,7105,0.638527,0.755986,5
2873,7159,0.638527,0.574789,4
2607,7161,0.638527,0.677290,4
2431,7150,0.638527,0.569684,4
2421,7104,0.638527,0.701907,4
3090,7142,0.638527,0.604222,4
229,7113,0.638527,0.653243,4
1789,7138,0.618054,0.614451,4
1964,7147,0.613227,0.580760,4
